In [76]:
import math
import pickle as pkl

from tqdm import tqdm
tqdm.pandas()

import yaml
import pandas as pd
import numpy as np
from numpy import triu
from nltk.tokenize import word_tokenize, TweetTokenizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [3]:
import nltk 
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [4]:
# Read in the processed data and the processed inbound dataset 
processed_df = pd.read_pickle("../data/processed/processed_v2.pkl")
processed_inbound_extra = processed_df["clean_inbound_text"]
processed_inbound_extra.head()

4     different people have given different answers ...
5     way to drop the ball on customer service so pi...
6     i want my amazon payments account closed dm me...
9     yeah this is crazy were less than a week away ...
10    how about you guys figure out my xbox one x pr...
Name: clean_inbound_text, dtype: object

In [5]:
# Reset the index. Set it properly in this iteration 
processed_df = processed_df.reset_index(drop=True)
processed_inbound_extra = processed_inbound_extra.reset_index(drop=True)

In [6]:
# Check for some null values 
processed_df.isnull().sum()

inbound_text               0
author_id                  0
created_at                 0
outbound_text              0
response_tweet_id      60327
inbound_lang               0
inbound_hashtags           0
outbound_hashtags          0
clean_inbound_text         0
clean_outbound_text        0
outbound_tokens_pos        0
inbound_tokens_pos         0
dtype: int64

In [7]:
# Read in the intents back 
with open(r"../objects/intents_amazon_support.yml") as file:
    intents = yaml.load(file, Loader=yaml.FullLoader)

# Previewing
print(f'\nintents:\n{intents}')
print(f'\nprocessed:\n{processed_df.head()}')


intents:
{'track': ['tracking', 'order', 'shipment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details'], 'support': ['service'], 'quality': ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different'], 'discount': ['prime', 'product', 'offer', 'price', 'sale'], 'account': ['email', 'orders', 'details', 'bank', 'access']}

processed:
                                        inbound_text   author_id  \
0  @AmazonHelp 3 different people have given 3 di...  AmazonHelp   
1  Way to drop the ball on customer service @1158...  AmazonHelp   
2  @115823 I want my amazon payments account CLOS...  AmazonHelp   
3  @AmazonHelp @115826 Yeah this is crazy we’re l...  AmazonHelp   
4  @115828 How about you guys figure out my Xbox ...  AmazonHelp   

                  created_at  \
0  2017-10-31 23:28:00+00:00   
1  2017-10-31 22:29:00+00:00   
2  2017-10-31 22:28:34+00:00   
3  2017-11-01 12

## Tweet Collection with Doc2Vec 
I can use my Doc2Vec representation to find top 1000 Tweets most similar to a generalized intent version of a Tweet based on it's cosine similarity. 

Heuristic search refers to a search strategy that attempts to optimize a problem by iteratively improving the solution based on a given heuristic function or a cost measure. My cost measure is trying to get the closest cosine distances.

So I basically trained my doc2vec model with my training data, which is the `processed_inbound`. I can actually compute a vector based on my training data to vectorize that word.

## Training my Doc2Vec 

### Data Synthesis 
Basically, there are 2 ways I can get my current training data (1000 for each)
* **Doc2Vec:** Some intent examples I will synthetically generate from an idealized example using doc2vec
* **Manual:** Some intent examples I will synthetically generate by duplicating and manual (like greeting, because the current data does not represent this)
* **Hybrid:** Some intents I will do a hybrid approach, where 50 percent might be my generated data, and 50 percent might be 

In [82]:
#TODO: Add information about Doc2Vec or post a link to it over here 

In [8]:
# Creating a synthetic dataset - generating N Tweets resembling a mock tweet
# This will subsequently be merged with the existing inbound data for inclusion in the doc2vec training process

# Version 1 - will be improved in future iterations
ideal = {
            "order track": "@AmazonHelo Hi, could you provide an update on the order? Its been days since the product has moved from its last location ", # change intent to "order tracking"???
            "product inquiry": "@AmazonHelp Looking for more info on the product. Can you share details or direct me to a reliable source?", # product inquire??? 
            "return refund": "@AmazonHelp How can I start a return process? The item I received doesn't match the description.",
            "account management": "@AmazonHelp Hi, I am having trouble logging into my account. Can you help me reset my password?", 
            "promotion discount": "@AmazonHelp Are there any ongoing promotions or deals in the ongoing festive season? Looking to buy a few items.",
            "shipping": "@AmazonHelp Hi, My address has changed. Can you help me update the shipping address for my order?",
            "technical support": "@AmazonHelp Encountering errors during checkout. Can you help me troubleshoot the issue?",
            "payment issue": "@AmazonHelp My payment method isn't going through. Any suggestions on how to resolve this?",
            "general query": "@AmazonHelp Hi, I have a general question regarding the product. Can you help me with this?"
        }

# Version 2 - Here I will try writing some more intent items 
ideal_tag_dict = {
                    "track": "tracking order shipoment late status carrier update number info received details", 
                    "support": "support chat customer resolution feedback satisfaction",
                    "quality": "quality product damaged received refund return issue order packaging proper working expected different", 
                    "discount": "prime product offer price sale", 
                    "account": "email orders details bank access"
                }

In [9]:
def add_extra(clean_inbound_text, extra_tweets):
    '''Adding extra tweets to the current tokenized data'''
    
    # Convert the extra tweets into a pandas Series
    extra_tweets = pd.Series(extra_tweets)

    ## The following 2 steps aren't required for now as the data is already in string format 
    ## Convert the extra tweets into a single string
    # print("Converting to string...")
    # string_processed_data = current_tokenized_data.progress_apply(" ".join)

    # Concatenate the extra tweets to the current data
    clean_inbound_text = pd.concat([clean_inbound_text, extra_tweets], axis = 0, ignore_index = True)

    # Tokenize the combined data
    tknzr = TweetTokenizer(strip_handles = True, reduce_len = True)
    print("Tokenizing...")
    tokenized_data = clean_inbound_text.progress_apply(tknzr.tokenize)
    string_processed_data = tokenized_data.progress_apply(" ".join)

    return string_processed_data

# Add the extra tweets to the current data
processed_inbound_extra = add_extra(processed_df["clean_inbound_text"], list(ideal_tag_dict.values()))

# Save the updated data to a pickle file
processed_inbound_extra.to_pickle("../objects/processed_inbound_extra.pkl")

processed_inbound_extra

Tokenizing...


100%|██████████| 122345/122345 [00:00<00:00, 2073258.14it/s]


0         different people have given different answers ...
1         way to drop the ball on customer service so pi...
2         i want my amazon payments account closed dm me...
3         yeah this is crazy were less than a week away ...
4         how about you guys figure out my xbox one x pr...
                                ...                        
122340    tracking order shipoment late status carrier u...
122341    support chat customer resolution feedback sati...
122342    quality product damaged received refund return...
122343                       prime product offer price sale
122344                     email orders details bank access
Length: 122345, dtype: object

In [10]:
processed_inbound_extra[-7:]

122338    no exchange available for i need to exchange m...
122339    there should be bonus and gifts for regular cu...
122340    tracking order shipoment late status carrier u...
122341    support chat customer resolution feedback sati...
122342    quality product damaged received refund return...
122343                       prime product offer price sale
122344                     email orders details bank access
dtype: object

In [11]:
processed_inbound_extra.shape

(122345,)

In [103]:
# Train a Doc2Vec model on the entire corpus 
def train_doc2vec(string_data, max_epochs, vec_size, alpha):
     
    # Tagging each of the documents with a unique ID
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(string_data)]
    
    # Instantiating my model 
    model = Doc2Vec(vector_size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm=1) # dm=1 means 'distributed memory' (PV-DM)
    
    # Building the vocabulary table
    model.build_vocab(tagged_data)
    
    for epoch in range(max_epochs): # Run for max_epochs
        print('iteration {0}'.format(epoch))    
        model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs) # This statement trains the model on the current epoch
        # Decreasing the learning rate
        model.alpha -= 0.0002
        # Fixing the learning rate, no decay
        model.min_alpha = model.alpha
        
    # Saving model
    model.save("../models/d2v.model")
    print("Model Saved")        
    
# Training
train_doc2vec(processed_inbound_extra, max_epochs=100, vec_size=20, alpha=0.025)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [12]:
# Loading in the Doc2Vec model 
model = Doc2Vec.load("../models/d2v.model")

# Storing all inbound data into a list for clustering 
inbound_d2v = np.array([model.infer_vector(word_tokenize(_d.lower())) for _d in list(processed_inbound_extra)])

# Saving
with open("../objects/inbound_d2v.pkl", "wb") as f:
    pkl.dump(inbound_d2v, f)

In [13]:
inbound_d2v.shape

(122345, 20)

Before, we did not have a concept of distance in our vectorizers, they don't really have a specific meaning. This is a much better way because it captures the contextual representations between words! Now the clustering should be a lot better than tfidf or bag of words.

## Methodology

Initially, to get the top 1000 similar Tweets, I tried using the existing data. But I don't think that would yield the most accurate results because I am not capturing not the best representative Tweet for an intent. For that reason, I made all these base representative Tweets myself (as seen in the values derived from `ideal` dict above). The goal is to find trying to find an idealized, wholistic representation of an intent. Then from there I use my doc2vec representations to find the top 1000 tweets most similar based on cosine similarity.

### Package Exploration

In [14]:
intents

{'track': ['tracking',
  'order',
  'shipment',
  'late',
  'status',
  'carrier',
  'update',
  'number',
  'info',
  'received',
  'details'],
 'support': ['service'],
 'quality': ['quality',
  'product',
  'damaged',
  'received',
  'refund',
  'return',
  'issue',
  'order',
  'packaging',
  'proper',
  'working',
  'expected',
  'different'],
 'discount': ['prime', 'product', 'offer', 'price', 'sale'],
 'account': ['email', 'orders', 'details', 'bank', 'access']}

In [15]:
## Trying an example 
## Finding and making idealized versions of each tweet so that I can 
intents_eg = {"discount": ["prime", "product", "offer", "price", "sale"]}
inferred_vectors = []

for keywords in intents_eg.values():
    inferred_vectors.append(model.infer_vector(intents_eg))
    
inferred_vectors

[array([ 0.125169  ,  0.05404945, -0.02987215, -0.02116698, -0.01440482,
        -0.02536038,  0.04863748, -0.00279213,  0.11392495,  0.02211997,
        -0.00445799, -0.07858939, -0.02147577, -0.0238472 , -0.04389124,
         0.12163858, -0.02892611,  0.02873815, -0.0653816 , -0.00789956],
       dtype=float32)]

In [205]:
#TODO: My Doc2Vec model vector is inferring different vector values for a given sample. I think I need to treat the model in a better fashion later on  

In [16]:
intents.values()

dict_values([['tracking', 'order', 'shipment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details'], ['service'], ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different'], ['prime', 'product', 'offer', 'price', 'sale'], ['email', 'orders', 'details', 'bank', 'access']])

In [17]:
# These are the current intents I wish to add to my training data 
print(intents) 

# Concatenate all the intent values in a sentence form 
ideal_values = list(" ".join(_words) for _words in intents.values())


{'track': ['tracking', 'order', 'shipment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details'], 'support': ['service'], 'quality': ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different'], 'discount': ['prime', 'product', 'offer', 'price', 'sale'], 'account': ['email', 'orders', 'details', 'bank', 'access']}


I have chosen my intent value tokens based on the frequency count of words as described in the previous notebooks. If performance is not upto the mark, reiterate and improve upon them in the future version(s)

### Finding intent tags 
I want to get the tags of my representative Tweets because that's what doc2vec's `model.similarity` method takes in as paramater to generate top N Tweets similar to it.

In [206]:
#TODO: Improve this block of code later - The following code block isn't the most efficient one

In [18]:
intents.items()

dict_items([('track', ['tracking', 'order', 'shipment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details']), ('support', ['service']), ('quality', ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different']), ('discount', ['prime', 'product', 'offer', 'price', 'sale']), ('account', ['email', 'orders', 'details', 'bank', 'access'])])

In [19]:
tokenizer = TweetTokenizer(strip_handles = True, reduce_len = True)

intents_repr = {k: tokenizer.tokenize(v) for k, v in ideal_tag_dict.items()}
print(intents_repr)

# Save them into objects 
with open("../objects/intents_repr.yml", "w") as f:
    yaml.dump(intents_repr, f, default_flow_style=False)

# Storing tags in order of the dictionary above
# tags = []

# Tokenize and process inbound tweets 
tokenized_processed_inbound_extra = processed_inbound_extra.apply(tokenizer.tokenize)

{'track': ['tracking', 'order', 'shipoment', 'late', 'status', 'carrier', 'update', 'number', 'info', 'received', 'details'], 'support': ['support', 'chat', 'customer', 'resolution', 'feedback', 'satisfaction'], 'quality': ['quality', 'product', 'damaged', 'received', 'refund', 'return', 'issue', 'order', 'packaging', 'proper', 'working', 'expected', 'different'], 'discount': ['prime', 'product', 'offer', 'price', 'sale'], 'account': ['email', 'orders', 'details', 'bank', 'access']}


In [20]:
processed_df

,inbound_text,author_id,created_at,outbound_text,response_tweet_id,inbound_lang,inbound_hashtags,outbound_hashtags,clean_inbound_text,clean_outbound_text,outbound_tokens_pos,inbound_tokens_pos
0,@AmazonHelp 3 different people have given 3 di...,AmazonHelp,2017-10-31 23:28:00+00:00,@115820 We'd like to take a further look into ...,619,en,[],[],different people have given different answers ...,wed like to take a further look into this with...,"[-PRON-: NOUN, d: VERB, like: VERB, to: NOUN, ...","[different: NOUN, people: NOUN, have: NOUN, gi..."
1,Way to drop the ball on customer service @1158...,AmazonHelp,2017-10-31 22:29:00+00:00,@115820 I'm sorry we've let you down! Without ...,616,en,[],[],way to drop the ball on customer service so pi...,i am sorry we have let you down without provid...,"[i: NOUN, be: NOUN, sorry: NOUN, -PRON-: NOUN,...","[way: NOUN, to: NOUN, drop: VERB, the: NOUN, b..."
2,@115823 I want my amazon payments account CLOS...,AmazonHelp,2017-10-31 22:28:34+00:00,@115822 I am unable to affect your account via...,NaN,en,[],[],i want my amazon payments account closed dm me...,i am unable to affect your account via twitter...,"[i: NOUN, be: NOUN, unable: NOUN, to: NOUN, af...","[i: NOUN, want: VERB, -PRON-: NOUN, amazon: NO..."
3,@AmazonHelp @115826 Yeah this is crazy we’re l...,AmazonHelp,2017-11-01 12:53:34+00:00,@115827 Thanks for your patience. ^KM,NaN,en,[],[],yeah this is crazy were less than a week away ...,thanks for your patience km,"[thank: NOUN, for: NOUN, -PRON-: NOUN, patienc...","[yeah: NOUN, this: NOUN, be: NOUN, crazy: NOUN..."
4,@115828 How about you guys figure out my Xbox ...,AmazonHelp,2017-10-31 22:28:00+00:00,@115826 I'm sorry for the wait. You'll receive...,627,en,[],[],how about you guys figure out my xbox one x pr...,i am sorry for the wait you will receive an em...,"[i: NOUN, be: NOUN, sorry: NOUN, for: NOUN, th...","[how: NOUN, about: NOUN, -PRON-: NOUN, guy: NO..."
...,...,...,...,...,...,...,...,...,...,...,...,...
122335,@AmazonHelp I sent you guys a DM regarding the...,AmazonHelp,2017-11-22 00:17:00+00:00,@328597 We're unable to access customer accoun...,NaN,en,[],[],i sent you guys a dm regarding the status of m...,were unable to access customer accounts via so...,"[be: NOUN, unable: NOUN, to: NOUN, access: VER...","[i: NOUN, send: VERB, -PRON-: NOUN, guy: NOUN,..."
122336,This is happening in my area w/@115821 “Prime”...,AmazonHelp,2017-11-22 02:16:55+00:00,"@777901 I'm sorry for the delay, Brenda! We st...",2987557,en,[],[],this is happening in my area w prime deliverie...,i am sorry for the delay brenda we strive to s...,"[i: NOUN, be: NOUN, sorry: NOUN, for: NOUN, th...","[this: NOUN, be: NOUN, happen: VERB, in: NOUN,..."
122337,@132994 @132995 @115850 got my #OnePlus5T at 8...,AmazonHelp,2017-11-22 03:49:29+00:00,@823783 Woohoo! That's awesome! Hope you love ...,2987674,en,"[#AmazonPrime, #OnePlus5T]",[],got my at am thanks for fulfilling the order fast,woohoo that is awesome hope you love the phone js,"[woohoo: NOUN, that: NOUN, be: NOUN, awesome: ...","[get: VERB, -PRON-: NOUN, at: NOUN, be: NOUN, ..."
122338,@115850 @132994 No exchange available for #One...,AmazonHelp,2017-11-22 05:22:31+00:00,@823802 The Exchange Offer is currently availa...,NaN,en,[#OnePlus5T],[],no exchange available for i need to exchange m...,the exchange offer is currently available only...,"[the: NOUN, exchange: NOUN, offer: NOUN, be: N...","[no: NOUN, exchange: NOUN, available: NOUN, fo..."


In [22]:
# processed_inbound_extra = pd.concat([processed_inbound_extra, pd.Series(ideal_tag_dict.values())], axis=0, ignore_index=True)

In [ ]:
# tokenized_processed_inbound_extra = pd.concat([tokenized_processed_inbound_extra, pd.Series(intents_repr.values())], axis=0, ignore_index=True)

In [28]:
processed_inbound_extra 

0         different people have given different answers ...
1         way to drop the ball on customer service so pi...
2         i want my amazon payments account closed dm me...
3         yeah this is crazy were less than a week away ...
4         how about you guys figure out my xbox one x pr...
                                ...                        
122340    tracking order shipoment late status carrier u...
122341    support chat customer resolution feedback sati...
122342    quality product damaged received refund return...
122343                       prime product offer price sale
122344                     email orders details bank access
Length: 122345, dtype: object

In [29]:
tokenized_processed_inbound_extra

0         [different, people, have, given, different, an...
1         [way, to, drop, the, ball, on, customer, servi...
2         [i, want, my, amazon, payments, account, close...
3         [yeah, this, is, crazy, were, less, than, a, w...
4         [how, about, you, guys, figure, out, my, xbox,...
                                ...                        
122340    [tracking, order, shipoment, late, status, car...
122341    [support, chat, customer, resolution, feedback...
122342    [quality, product, damaged, received, refund, ...
122343                 [prime, product, offer, price, sale]
122344               [email, orders, details, bank, access]
Length: 122345, dtype: object

In [21]:
# Find the index locations of specific Tweets
# get_indices = []
# Storing tags in order of the dictionary above
tags = []
def report_index_loc(tweet, intent_name):
    ''' Takes in the tweet to find the index for and returns a report of that tweet index along with what the 
    representative Tweet looks like'''
    try:
        tweets = [] # List which stores tuples of indexes of representative tweets AND a boolean value to indicate if the tweet has the intent we are looking for
        for i, j in enumerate(tokenized_processed_inbound_extra):
            if j == tweet:
                tweets.append((i, True))
            else:
                tweets.append((i, False))

        indices = [i[0] for i in tweets if i[1] == True]
        # get_indices.append(indices.append(i[0]) if i[1] == True else False for i in tweets)

        preview = processed_inbound_extra.iloc[indices]

        # Appending to indexes for dictionary 
        tags.append(str(indices[0]))
        print(tags)
        

    except IndexError as e:
        print("Index not in list, move on")
        return

    return intent_name, str(indices[0]), preview


# Reporting and storing indexes with the function
print("TAGGED INDEXES TO LOOK FOR")
for j, i in intents_repr.items():
    try:
        print('\n{} \nIndex: {}\nPreview: {}'.format(*report_index_loc(i, j)))
    except Exception as e:
        print("Index ended")

TAGGED INDEXES TO LOOK FOR
['122340']

track 
Index: 122340
Preview: 122340    tracking order shipoment late status carrier u...
dtype: object
['122340', '122341']

support 
Index: 122341
Preview: 122341    support chat customer resolution feedback sati...
dtype: object
['122340', '122341', '122342']

quality 
Index: 122342
Preview: 122342    quality product damaged received refund return...
dtype: object
['122340', '122341', '122342', '122343']

discount 
Index: 122343
Preview: 122343    prime product offer price sale
dtype: object
['122340', '122341', '122342', '122343', '122344']

account 
Index: 122344
Preview: 122344    email orders details bank access
dtype: object


In [43]:
tokenized_processed_inbound_extra

0         [different, people, have, given, different, an...
1         [way, to, drop, the, ball, on, customer, servi...
2         [i, want, my, amazon, payments, account, close...
3         [yeah, this, is, crazy, were, less, than, a, w...
4         [how, about, you, guys, figure, out, my, xbox,...
                                ...                        
122340    [tracking, order, shipoment, late, status, car...
122341    [support, chat, customer, resolution, feedback...
122342    [quality, product, damaged, received, refund, ...
122343                 [prime, product, offer, price, sale]
122344               [email, orders, details, bank, access]
Length: 122345, dtype: object

In [61]:
inferred_vector = model.infer_vector(tokenized_processed_inbound_extra[0])
inferred_vector

array([ 0.19001217,  0.48508617,  0.4193148 , -0.12760295, -0.01929441,
       -0.35454088,  0.21225552, -0.01340768,  1.4679474 ,  0.19322701,
        0.07695433, -0.6741293 , -0.16881841,  0.0572688 ,  0.29471058,
       -0.12437083, -0.6316176 ,  0.06095588, -0.2637178 , -0.19627158],
      dtype=float32)

In [62]:
# Great! Now I can get the training data for my battery intent (as an example)
similar_doc = model.dv.most_similar([inferred_vector], topn = 1000)
# Preview
similar_doc

[('119157', 0.9542658925056458),
 ('55898', 0.9530379772186279),
 ('39889', 0.9519811272621155),
 ('120356', 0.9518635272979736),
 ('64693', 0.9491467475891113),
 ('88323', 0.9467576146125793),
 ('20366', 0.943752646446228),
 ('109260', 0.9338643550872803),
 ('89491', 0.9313428997993469),
 ('27960', 0.9297406673431396),
 ('47042', 0.9278128743171692),
 ('110014', 0.9224398136138916),
 ('47382', 0.9113422632217407),
 ('101151', 0.9058321118354797),
 ('115065', 0.9031236171722412),
 ('78397', 0.898678719997406),
 ('28241', 0.8983146548271179),
 ('108069', 0.8921398520469666),
 ('45208', 0.8802627921104431),
 ('22731', 0.8800346255302429),
 ('49616', 0.879146933555603),
 ('82055', 0.8790873885154724),
 ('49615', 0.8778648972511292),
 ('84653', 0.876183807849884),
 ('55072', 0.875387966632843),
 ('134', 0.8752726912498474),
 ('52362', 0.8752098679542542),
 ('85208', 0.8742002844810486),
 ('54692', 0.8725416660308838),
 ('85142', 0.8721601366996765),
 ('75460', 0.8720210790634155),
 ('81977

In [52]:
tokenized_processed_inbound_extra[0]

['different',
 'people',
 'have',
 'given',
 'different',
 'answers',
 'and',
 'i',
 'still',
 'do',
 'not',
 'have',
 'my',
 'order',
 'says',
 'delivered',
 'saturday',
 'was',
 'not',
 'i',
 'was',
 'home',
 'all',
 'day']

In [67]:
tokenized_processed_inbound_extra

0         [different, people, have, given, different, an...
1         [way, to, drop, the, ball, on, customer, servi...
2         [i, want, my, amazon, payments, account, close...
3         [yeah, this, is, crazy, were, less, than, a, w...
4         [how, about, you, guys, figure, out, my, xbox,...
                                ...                        
122340    [tracking, order, shipoment, late, status, car...
122341    [support, chat, customer, resolution, feedback...
122342    [quality, product, damaged, received, refund, ...
122343                 [prime, product, offer, price, sale]
122344               [email, orders, details, bank, access]
Length: 122345, dtype: object

In [72]:
tokenized_processed_inbound_extra[49615]

['any', 'update']

In [74]:
tokenized_processed_inbound_extra[109260]

['last', 'night']

In [ ]:
#TODO: Data Cleaning pe dhyan dena pdega. Abhi sahi nai hai 

In [28]:
processed_inbound_extra[122340]

'tracking order shipoment late status carrier update number info received details'

In [29]:
# TODO: Processed inbound extra, there are spelling issues though which needs to be resolved

In [50]:
tokenized_processed_inbound_extra[122340]

['tracking',
 'order',
 'shipoment',
 'late',
 'status',
 'carrier',
 'update',
 'number',
 'info',
 'received',
 'details']

In [57]:
tags

['122340', '122341', '122342', '122343', '122344']

In [53]:
intents_repr.keys()

dict_keys(['track', 'support', 'quality', 'discount', 'account'])

In [31]:
# Create the intent-tag dictionary by # Dictionary mapping the intent to the row index 
intents_tags = dict(zip(intents_repr.keys(), tags))
intents_tags

{'track': '122340',
 'support': '122341',
 'quality': '122342',
 'discount': '122343',
 'account': '122344'}

## Data Synthesis 
### 1. Adding intents to training data based on similarity 
As can be seen above, the right tuple element is the cosine similarity. We are just taking the top 1000 similar to a base, idealized version of intents (which we based off keywords mostly).

### 2. Adding the intents manually 
These ones are generated with a different method that is more manual. I will generate as much examples of this as I can, then I brute force it by duplicating it until it reaches 1000 training examples for preserving class balance.

### 3. Adding in the hybrid intents 
I use my Keyword Exploration shown in the previous notebook and find that there is a lot of overlap between **track** and **quality**. So for both I am going to generate a proportion of it using doc2vec, and the rest I will manually insert examples - the idea is to balance overfitting or noise and putting in the correct signal.

In [37]:
#TODO: **One special case might be out of scope, I might get an alternative way to deal with that since I might not be able to generate all examples of that intent.**
#TODO: **sSee if I might run into this issue**

Here's is the catch. Have intent buckets inplace already in place. You would need to supply this to the below function to get your top N tweets corresponding to the current tweet


## Checking for stopwords because I don't want to include them in the manually represented intents. 
## This is something I manually tune to the 

## Check for stopwords since I don't want to add them in the manually represented intents 
## This is something I would manually tune to the dataframe (for step 2 of the process)


In [78]:
## Getting top n tweets similar to the first tweet 
# This will return a list of tuples (i, j) where i is the index and j is the 
# cosine similarity to the tagged document index 

# Storing all intents in a dataframe 
train = pd.DataFrame()

# Storing tags in order of the dictionary above 
def generate_intent(target, idx_tag): 
    '''Function that maps an index tag to an intent and returns nsim number of similar tweets'''
    sim_docs = model.dv.most_similar(idx_tag, topn = target)
    
    # Getting just the indexes 
    indexes = [int(i[0]) for i in sim_docs]
    
    # Actually seeing the top 1000 tweets similar to 0th tweet which seems to be about updates 
    # print(processed_inbound_extra[indexes])
    # Adding just the values, AND not the index 
    # Tokenize the tweet 
    return [word_tokenize(tweet) for tweet in list(processed_inbound_extra[indexes].values)]

# Update training data 
for intent, tag in intents_tags.items():
    print('Intent: ', intent)
    train[intent] = generate_intent(1000, tag)
    print('\n')


# 2. Manually added intents 
# These are the remainder intents 
manually_added_intents = {
    "speak_representative": [["talk", "human", "please"], 
                             ['let','me','talk','to','apple','support'], 
                             ['can','i','speak','agent','person']], 
    "greeting": [["hi"], ["hello"], ["whats", "up"], ["good", "morning"], ["good", "afternoon"], ["good", "evening"], ["good", "night"]],
    "goodbye": [["goodb"],["bye"], ["thank"], ["thanks"], ["done"]], 
    "challenge_robot": [["robot", "human"], ["are", "you", "robot"],
                       ['who','are','you']]

}

# Insert manually added intents to data 
def insert_manually(target, prototype): 
    ''' Taking a prototype tokenized document to repeat until I get desired length target'''
    factor = math.ceil(target / len(prototype))
    content = prototype * factor
    return [content[i] for i in range(target)]

# Update training data
for intent_name in manually_added_intents.keys(): 
    train[intent_name] = insert_manually(1000, [*manually_added_intents[intent_name]])

# 3. Adding in the hybrid intents 
hybrid_intents = {
    "order track": ["track", "order", "shipment", "late", "status", "carrier", "update", "number", "info", "received", "details"],
    "product inquiry": ["product", "inquire", "details", "source", "reliable"],
    "return refund": ["return", "process", "item", "received", "match", "description"],
    "account management": ["trouble", "logging", "account", "reset", "password"],
    "promotion discount": ["ongoing", "promotions", "deals", "festive", "season", "buy", "items"],
    "shipping": ["address", "changed", "update", "shipping", "address", "order"],
    "technical support": ["errors", "checkout", "troubleshoot", "issue"],
    "payment issue": ["payment", "method", "going", "through", "resolve"],
    "general query": ["general", "question", "product"]
}

# 3. Adding in the hybrid intents
hybrid_intents = {}


def insert_hybrid(manual_target, generated_target, prototype, itag): 
    return insert_manually(manual_target, prototype) + generate_intent(generated_target, itag)

Intent:  track


Intent:  support


Intent:  quality


Intent:  discount


Intent:  account




In [ ]:
# TODO: I am not inserting the hybrid intents as of now, since I believe it might just add to unnecessary noise in the dataset. Also, its just complex in the first go. So definitely in 2nd iteration 

In [80]:
# Jut check what all intents are there 
train

,track,support,quality,discount,account,speak_representative,greeting,goodbye,challenge_robot
0,"[no, information]","[very, poor, feedback, very, disappointing, se...","[done, attached, is, the, proof, of, completion]","[amazon, pay, transaction, amt, ₹]","[email, account, details]","[talk, human, please]",[hi],[goodb],"[robot, human]"
1,"[issue, is, resolved, and, item, is, being, re...","[already, done, i, am, frankly, fed, up, with,...","[return, pick, up, not, happening]","[next, sale, date]","[email, account, details]","[let, me, talk, to, apple, support]",[hello],[bye],"[are, you, robot]"
2,"[expected, delivery, date, is, th, october, tr...","[very, poor, feedback, very, disappointing, se...","[your, target, completed, return, policy, expi...","[mrp, on, product, rs, mrp, on, site, rs]","[email, account, details]","[can, i, speak, agent, person]","[whats, up]",[thank],"[who, are, you]"
3,"[expected, delivery, date, is, th, october, tr...","[can, see, you, have, replied, to, others, who...","[order, is, lost, no, one, taking, responsibil...","[but, flipkart, and, shop, clues, are, giving,...","[the, credit, card, information, is, correct]","[talk, human, please]","[good, morning]",[thanks],"[robot, human]"
4,"[no, emails, no, reason, for, delay]","[my, issue, is, not, resolved, really, should,...","[tampered, supplied, defective, fake, failedde...","[but, flipkart, and, shop, clues, are, giving,...","[account, email, email]","[let, me, talk, to, apple, support]","[good, afternoon]",[done],"[are, you, robot]"
...,...,...,...,...,...,...,...,...,...
995,"[details, shared]","[did, you, know, amazon, india, fools, people,...","[pathetic, service, recvd, used, phone, wo, ac...","[you, are, not, able, to, help, me, so, far, w...","[i, am, trying, to, call, customer, service, f...","[can, i, speak, agent, person]",[hello],[goodb],"[who, are, you]"
996,"[i, could, not, find, any, correspondence, in,...","[that, website, does, not, explain, anything, ...","[most, ridiculous, packaging, i, have, seen, f...","[here, is, the, latest, update, this, item, wa...","[hey, i, order, tons, of, stuff, from, you, am...","[talk, human, please]","[whats, up]",[bye],"[robot, human]"
997,"[thanks, for, reply, guest, has, left, this, i...","[happy, to, state, that, the, folks, have, com...","[form, filled, up, do, the, needful]","[i, bought, sony, headsetits, mic, not, upto, ...","[does, amazon, no, longer, provide, refund, if...","[let, me, talk, to, apple, support]","[good, morning]",[thank],"[are, you, robot]"
998,"[my, order, number, is]","[pathetic, after, sales, serviceampinstallatio...","[is, there, any, reason, my, order, required, ...","[fourth, time, in, a, row, i, feel, cheated, b...","[why, address, verification, team, does, not, ...","[can, i, speak, agent, person]","[good, afternoon]",[thanks],"[who, are, you]"


,Utterance,Intent
0,"[no, information]","[issue, is, resolved, and, item, is, being, re..."
1,"[very, poor, feedback, very, disappointing, se...","[already, done, i, am, frankly, fed, up, with,..."
2,"[done, attached, is, the, proof, of, completion]","[return, pick, up, not, happening]"
3,"[amazon, pay, transaction, amt, ₹]","[next, sale, date]"
4,"[email, account, details]","[email, account, details]"


In [98]:
# Save the training data to a serialized file
train.to_pickle("../objects/train.pkl")

## Style the display 
show = lambda x: x.head(10).style.set_properties(**{'background-color': 'black',                                                   
                                    'color': 'lawngreen',                       
                                    'border-color': 'white'})\
.map(lambda x: f"color: {'lawngreen' if isinstance(x,str) else 'red'}")\
.background_gradient(cmap='Blues')

print(train.shape)
show(train)

(1000, 9)


,track,support,quality,discount,account,speak_representative,greeting,goodbye,challenge_robot
0,"['no', 'information']","['very', 'poor', 'feedback', 'very', 'disappointing', 'services']","['done', 'attached', 'is', 'the', 'proof', 'of', 'completion']","['amazon', 'pay', 'transaction', 'amt', '₹']","['email', 'account', 'details']","['talk', 'human', 'please']",['hi'],['goodb'],"['robot', 'human']"
1,"['issue', 'is', 'resolved', 'and', 'item', 'is', 'being', 'redelivered']","['already', 'done', 'i', 'am', 'frankly', 'fed', 'up', 'with', 'the', 'support', 'you', 'replies', 'are', 'template', 'replies']","['return', 'pick', 'up', 'not', 'happening']","['next', 'sale', 'date']","['email', 'account', 'details']","['let', 'me', 'talk', 'to', 'apple', 'support']",['hello'],['bye'],"['are', 'you', 'robot']"
2,"['expected', 'delivery', 'date', 'is', 'th', 'october', 'tracking', 'number']","['very', 'poor', 'feedback', 'very', 'disappointing', 'services']","['your', 'target', 'completed', 'return', 'policy', 'expired', 'how', 'much', 'time', 'required', 'now', 'closed', 'this', 'issue', 'disappointed', 'this', 'type', 'of', 'service']","['mrp', 'on', 'product', 'rs', 'mrp', 'on', 'site', 'rs']","['email', 'account', 'details']","['can', 'i', 'speak', 'agent', 'person']","['whats', 'up']",['thank'],"['who', 'are', 'you']"
3,"['expected', 'delivery', 'date', 'is', 'th', 'october', 'tracking', 'number']","['can', 'see', 'you', 'have', 'replied', 'to', 'others', 'who', 'tweeted', 'after', 'me', 'i', 'am', 'a', 'v', 'unhappy', 'prime', 'customer', 'onus', 'on', 'me', 'to', 'go', 'out', 'of', 'my', 'way', 'to', 'return', 'item', 'when', 'wrong', 'item', 'was', 'sent']","['order', 'is', 'lost', 'no', 'one', 'taking', 'responsibility', 'no', 'follow', 'up']","['but', 'flipkart', 'and', 'shop', 'clues', 'are', 'giving', 'cod', 'facility', 'in', 'this', 'pincode']","['the', 'credit', 'card', 'information', 'is', 'correct']","['talk', 'human', 'please']","['good', 'morning']",['thanks'],"['robot', 'human']"
4,"['no', 'emails', 'no', 'reason', 'for', 'delay']","['my', 'issue', 'is', 'not', 'resolved', 'really', 'should', 'stop', 'business', 'in', 'india']","['tampered', 'supplied', 'defective', 'fake', 'faileddetectonline', 'help', 'identified', 'fake', 'cartridges']","['but', 'flipkart', 'and', 'shop', 'clues', 'are', 'giving', 'cod', 'facility', 'in', 'this', 'pincode']","['account', 'email', 'email']","['let', 'me', 'talk', 'to', 'apple', 'support']","['good', 'afternoon']",['done'],"['are', 'you', 'robot']"
5,"['provided', 'info']","['everyday', 'following', 'up', 'with', 'you', 'are', 'mail', 'support', 'for', 'past', 'days', 'they', 'kept', 'me', 'on', 'wait', 'and', 'from', 'last', 'couple', 'of', 'days', 'they', 'are', 'not', 'even', 'responding', 'to', 'the', 'mails', 'such', 'a', 'worthless', 'service', 'no', 'one', 'cares']","['tampered', 'supplied', 'defective', 'fake', 'faileddetectonline', 'help', 'identified', 'fake', 'cartridges']","['order', 'ordered', 'on', 'oct', 'product', 'is', 'prime', 'eligible', 'i', 'have', 'prime', 'membership', 'still', 'after', 'days', 'not', 'shipped']","['my', 'account', 'login', 'is', 'email']","['can', 'i', 'speak', 'agent', 'person']","['good', 'evening']",['goodb'],"['who', 'are', 'you']"
6,"['order', 'was', 'to', 'be', 'delivered', 'yesterday', 'had', 'no', 'intimation', 'of', 'change', 'amazon', 'team', 'went', 'ahead', 'and', 'updated', 'rescheduled', 'on', 'customer', 'request']","['i', 'have', 'already', 'written', 'item', 'delivery', 'feedback', 'but', 'no', 'response', 'you', 'have', 'waste', 'rural', 'service', 'than', 'others']","['worse', 'buying', 'experience', 'from', 'returned', 'awb', 'new', 'phone', 'not', 'yet', 'dispatched']","['ordered', 'product', 'on', 'prime', 'but', 'product', 'delivery', 'is', 'informed', 'after', 'days']","['account', 'email', 'email']","['talk', 'human', 'please']","['good', 'night']",['bye'],"['robot', 'human']"
7,"['hey', 'are', 'you', 'experiencing', 'i

I am quite pleased with these. They seem quite promising if I inspect them!!!. Just don't worry too much about the preprocessing thing and stuff. I just want my model to be generalizable overall... 

In [ ]:
# Convert to long dataframe format from wide so my data can read it 
neat_train = pd.DataFrame(train.T.unstack()).reset_index().iloc[:,1:].rename(columns={"level_1":"Intent", 0: "Utterance"})

In [ ]:
# Intent classification with Keras 

